![<CW3E Logo>](https://cw3e.ucsd.edu/images/cw3e_logo_files/wetransfer-b4ff74/CW3E%20Final%20Logo%20Suite/4-Horizontal-Acronym/Digital/PNG/CW3E-Logo-Horizontal-Acronym-FullColor.png "Center for Western Weather and Water Extremes Logo")

# Creating a Singularity Container for a Micromamba Environment on Expanse

---

## Overview

The first few sections are covered by [Jozette's notebook](https://github.com/CW3E/Research-Code-Portability-Tutorial/blob/main/Creating-a-Singularity-Container-for-a-Conda-Environment.ipynb).

## Creating a Singularity Definition/Recipe File
---

Definition files (aka container recipes) are the blueprint for creating a custom Singularity container. These type of files consist of the **header** and the **sections**. These parts will be briefly discussed below. For more information on definition files, refer to the [Singularity Documentation](https://docs.sylabs.io/guides/latest/user-guide/definition_files.html).

This section significantly deviates from [Jozette's notebook](https://github.com/CW3E/Research-Code-Portability-Tutorial/blob/main/Creating-a-Singularity-Container-for-a-Conda-Environment.ipynb).

### Generating a Definition File

---

The following line of code can be run to create your own definition file. Make sure to have this definition file in a system that has the capability to build a container (i.e., skyriver or feather).

```
vim Singularity.def
```

> Usually, the definition file for a Singularity container is just named "Singularity". However, if you are making a container for a specific environment, make sure to name it appropriately.
>> Ex: `Singularity_NetCDF.def` as a definition file name to build a NetCDF environment container.

### Header

---

The header section of a definition file describes the core OS to build and configure within the container. The header can be comprised of several keywords, but for the purposes of our container, we will only need `Bootstrap` and `From`. An example header is shown and explained below.

```
Bootstrap: docker
From: mambaorg/micromamba:1.5.8
```

1. **Bootstrap** - this keyword refers to what kind of base you want to use (in this case, we want to use Docker).
2. **From** - this keyword refers to the named container/reference to layers you want to use. In this case, there's already a [micromamba-docker image](https://micromamba-docker.readthedocs.io/en/latest/) that we're going to use.

For more reference to the docker bootstrap agent, refer to the [Singularity appendix.](https://docs.sylabs.io/guides/latest/user-guide/appendix.html#build-docker-module)

### Sections

---

The main content of a definition file is within the sections. Each section has its own function and provides different content and commands. The following sections were used to create the micromamba Singularity container (in this respective order).

##### %labels

This section is used to store metadata within the container. This section is often filled with information about the author and application. An example `%label` section is shown below.

```
%labels
    APPLICATION1_NAME Micromamba NWP Env
    APPLICATION1_URL
    APPLICATION1_VERSION 1.0

    APPLICATION2_NAME micromamba-docker
    APPLICATION2_URL https://github.com/mamba-org/micromamba-docker
    APPLICATION2_VERSION 1.5.8

    AUTHOR_NAME Corrine DeCiampa
    AUTHOR_EMAIL cdeciampa@ucsd.edu

    CO_AUTHOR_NAME Colin Grudzien
    CO_AUTHOR_EMAIL cgrudzien@ucsd.edu

    CO_AUTHOR_NAME Jozette Conti
    CO_AUTHOR_EMAIL jlconti@ucsd.edu

    CO_AUTHOR_NAME Patrick Mulrooney
    CO_AUTHOR_EMAIL pmulrooney@ucsd.edu

    LAST_UPDATED 2024.07.18

```

##### %setup

Any commands in this section are executed on the host system outside the container after the base OS has been installed. For the purposes of the conda container, leave this blank.

```
%setup
```

##### %environment

This section allows you to define environmental variables set at runtime. If there are any variables needed during the build time, place them in the [`%post` section](#%post). During the building of the container, this section will be written to a file in the container’s metadata folder. This folder is then sourced during the runtime. For the purposes of the micromamba environment, it's extremely important that you bind these environment paths:

```
%environment
    # export generic path to bind work scripts / modules to
    export PATH="${MAMBA_ROOT_PREFIX}/bin:$PATH"
    export PYTHONPATH=${PYTHONPATH}:/scrpt_dir
```

##### %files

This section is significant for our container. In this section, you can copy any necessary files into the container. For the purposes of our micromamba container, we need to generate a list of requirements for the conda environment in Expanse. This can be achieved by running the following line in the base conda environment within Expanse, replacing `ENVIRONMENT_NAME` with the name of your conda environment.

```
conda env export --from-history --name ENVIRONMENT_NAME > ENVIRONMENT_NAME.yml
```

The `--from-history` flag tells conda to export a minimal list with the core package names specified when the environment was created:

```
name: nwp_env
channels:
  - conda-forge
dependencies:
  - python=3.12.1
  - numpy
  - pandas
  - matplotlib
  - seaborn
  - cartopy
  - jupyter
  - netcdf4
  - xarray
  - scipy
```

This is much more portable than exporting the full dependencies channel hash as those specific packages may be incompatible across different systems, which prevents micromamba from effectively resolving dependency conflicts. Using the `--from-history` flag can avoid dependency conflicts when trying to build a new conda environment from an exported `ENVIRONMENT_NAME.yml` file.

This `.yml` file, which is included in this repo as `nwp_env.yml`, can then be embedded into the container in the `/opt` folder through the definition file, as shown below.

```
%files
    # Import yml specs
    ./nwp_env.yml /opt
```

Now the required conda packages in Expanse are within our definition file.

<a id="%post"></a>
##### %post

The `%post` section is the main section of all the sections. Making directories and installing software/libraries occurs within this section. This section consists of commands that would be run if a root user was directly in the terminal. (i.e., this section are the commands needed to install your program successfully). 

Our `%post` section is as follows:

```
%post -c /bin/bash

    # Install into base environment from input yml
    micromamba install -y -n base -f /opt/nwp_env.yml && \
    micromamba clean --all --yes
    rm -f /opt/nwp_env.yml
```

Since we're using the [micromamba-docker image](https://hub.docker.com/r/mambaorg/micromamba), there's no need to download, install, and initialize micromamba. We can simply build our environment from the conda environment requirements `.yml` that was imported from Expanse.

After this, we are done with the `%post` section.

##### %runscript

This section comprises of the commands you would use to run the now-installed program from the `%post` section. 

The devs for the micromamba-docker image didn't build it with singularity in mind, so there is a small quirk that needs to be addressed in the runscript section.

```
%runscript
    #!/bin/bash

    # Activate environment at system profile level for exec/ run/ instance/ shell
    # provides Python libraries and all backends
    source /usr/local/bin/_activate_current_env.sh
```

This command in the `%runscript` section initializes `bash` and sources a script inherited from the `micromamba-docker` image that activates the base micromamba environment. Without this section, you would need to supply an extra command for `singularity exec` and `singularity run`.

##### %test

This is the last section of our definition file! This section is run at the end of the build process. The test section for our definition files is just

```
%test
```

#### Finalized Definition File
---

Now that we have added all the necessary information, our final definition file should look like the following.  

In [1]:
cat nwp_env.def

Bootstrap: docker
From: mambaorg/micromamba:1.5.8

%labels
    APPLICATION1_NAME Micromamba NWP Env
    APPLICATION1_URL 
    APPLICATION1_VERSION 1.0

    APPLICATION2_NAME micromamba-docker
    APPLICATION2_URL https://github.com/mamba-org/micromamba-docker
    APPLICATION2_VERSION 1.5.8

    AUTHOR_NAME Corrine DeCiampa
    AUTHOR_EMAIL cdeciampa@ucsd.edu

    CO_AUTHOR_NAME Colin Grudzien
    CO_AUTHOR_EMAIL cgrudzien@ucsd.edu

    CO_AUTHOR_NAME Jozette Conti
    CO_AUTHOR_EMAIL jlconti@ucsd.edu

    CO_AUTHOR_NAME Patrick Mulrooney
    CO_AUTHOR_EMAIL pmulrooney@ucsd.edu

    LAST_UPDATED 2024.07.18

%files
    # Import yml specs
    ./nwp_env.yml /opt

%environment
    # export generic path to bind work scripts / modules to
    export PATH="${MAMBA_ROOT_PREFIX}/bin:$PATH"
    export PYTHONPATH=${PYTHONPATH}:/scrpt_dir

%post -c /bin/bash 

    # Install environment at prefix
    micromamba install -y -n base -f /opt/nwp_env.yml && \
    micromamba clean --all --yes 
    rm -f /o

This is also included as the `nwp_env.def` file in this repo.

## Building a Singularity Container From a Definition File
---

This section is covered by [Jozette's notebook](https://github.com/CW3E/Research-Code-Portability-Tutorial/blob/main/Creating-a-Singularity-Container-for-a-Conda-Environment.ipynb).


## Running/Shelling into a Singularity Container
---

This section is covered by [Jozette's notebook](https://github.com/CW3E/Research-Code-Portability-Tutorial/blob/main/Creating-a-Singularity-Container-for-a-Conda-Environment.ipynb).

## Transferring a Singularity Container To Expanse

---

As Singularity containers cannot be built on Expanse, we must transfer the container we made on a different system to Expanse. We can run a rsync command (preferred over scp) to do this. The following syntax can be used for this command.

```
rsync -avh --progress /path/to/nwp_env.sif <whoami>@login.expanse.sdsc.edu:/path/to/where/you/want/nwp_env.sif
```

For example, the line of code I ran for this was, 

```
rsync -avh --progress /home/cdeciampa/containers/nwp_env.sif cdeciampa@login.expanse.sdsc.edu:/expanse/nfs/cw3e/cwp157/cdeciampa/SOFT_ROOT/nwp_env.sif
```

> This line of code is to be run in feather/skyriver. It then copies the Singularity image to Expanse.

* The `--progress` flag will show the user the process of the image transfer. Once completed, make sure to verify the image has, in fact, transferred to the desired directory in Expanse.

<a id="verify"></a>
## Verifying Your Container

---

Once the miniconda container has been transferred, run the following line of code</a> with your desired path in Expanse.

```
singularity exec -e -B /expanse:/expanse /path/to/nwp_env.sif /usr/local/bin/_entrypoint.sh micromamba list
```

For example, the line of code I ran was

```
singularity exec -e -B /expanse:/expanse /expanse/nfs/cw3e/cwp157/cdeciampa/SOFT_ROOT/nwp_env.sif /usr/local/bin/_entrypoint.sh micromamba list
```

The `-e` flag is a longer form of the `-cleanenv` flag. Including this flag helps make sure the container environment is clean of any inherited environment variables from the host system. Hence, for best practices, this flag should be included in any `singularity exec` command.

> When using a container, it acts to replace your home operating system. However, there are times when we use a container and still want to reference files/directories on the home system. The `-B` or `--bind` flag is used to set up user defined bind paths, binding directories from the host system to the directories within the container. The syntax for this binding is `/hostdirectory:/containerdirectory` and multiple bind paths can be written by separating with a comma.
>> If running this container on a server other than Expanse, these bind paths will need to be adjusted.

Once this command has been successfully run, you should see a list of all the installed packages in your container. Verify this list to ensure the container was built with the correct requirements.

### A quick note:

As mentioned earlier, the `micromamba-docker` image has that quirk where micromamba needs to be sourced in the `%%runscript` section before you build the singularity container. Having that section there means that you don't need to source micromamba to `run` or `exec` python scripts:

```
singularity exec -e -B /expanse:/expanse /path/to/nwp_env.sif python hello_world.py
```

The command above will use the conda environment in your singularity container to run a python script located on Expanse without any issues or extra steps.

However, if you want to call micromamba directly, you *do* need to include `/usr/local/bin/_entrypoint.sh` before the `micromamba` command. If you leave that part out of a command calling micromamba:

```
singularity exec -e -B /expanse:/expanse /path/to/nwp_env.sif micromamba list
```

This is what it returns:

```
List of packages in environment: ""

```

This doesn't mean that there aren't any packages installed in your conda environment in the singularity container. It's just a bug inherited from the `micromamba-docker` image where the `micromamba` command doesn't know where micromamba is. This extra command is **only** needed if you want to call micromamba (`micromamba list`). It's not necessary to include for anything else, including running standalone python scripts.

## Implementing the Container into Bash Scripts

---

This section is largely the same as [Jozette's notebook](https://github.com/CW3E/Research-Code-Portability-Tutorial/blob/main/Creating-a-Singularity-Container-for-a-Conda-Environment.ipynb), so I'll be skipping it.

The only difference is that the `--bind` command should be `--bind /expanse:/expanse` for Expanse

## Using Containers to Run Python Scripts
---

This section is largely the same as [Jozette's notebook](https://github.com/CW3E/Research-Code-Portability-Tutorial/blob/main/Creating-a-Singularity-Container-for-a-Conda-Environment.ipynb), so I'll be skipping it.

The only difference is that the `--bind` command should be `--bind /expanse:/expanse` for Expanse

## Using a Singularity Container to Launch a Jupyter Notebook
---

Another way we can utilize our container is by using it to launch a Juypter notebook server off of Expanse. Before we can achieve this, however, we need to check if our container has the required Jupyter packages. To check this, we can run the same line of code we used to [verify our container](#verify). For my specific container, I ran the following line 


```
singularity exec -e -B /expanse:/expanse /expanse/nfs/cw3e/cwp157/cdeciampa/SOFT_ROOT/nwp_env.sif /usr/local/bin/_entrypoint.sh micromamba list
```

If the Juypter `notebook` and/or `jupyterlab` packages are listed within the contained environment, then the container can be used to launch a Juypter notebook from the terminal using [galyleo](https://github.com/mkandes/galyleo/tree/master):

```
galyleo launch -A <project_ID> -p cw3e-shared -t 5-00:00:00 -j lab -B /expanse:/expanse,/scratch:/scratch --sif /expanse/path/to/nwp_env.sif -d /expanse/path/to/notebooks --env-modules singularitypro/4.1.2
```

Where:
  - `-p` is the partition, which is either `cw3e-shared` or `cw3e-compute`. For jupyter notebooks on Expanse, the partition should always be `cw3e-shared` since notebooks aren't nearly computationally heavy enough to require a full node.
  - `-t` is the time, which I chose 5 days.
  - `-j` is the choice between `notebook` and `lab`. JupyterLab does everything Jupyter Notebook does and much more.
  - `-B` are the same bindings as before, but now `/scratch:/scratch` is added as it's necessary for galyleo to run on Expanse.
  - `--sif` is the full path to the singularity container.
  - `-d` is the full path to where your Jupyter Notebook(s) are saved. This needs to be the path to a project folder, not your home or scratch directory.
  - `--env-modules` tells galyleo which modules are loaded at launch and galyleo will fail to launch unless `singularitypro/4.1.2` is included.
    - You also need to include slurm if you're going to be submitting jobs from your notebook:
      - `--env-modules singularitypro/4.1.2,slurm/expanse/current`

As an example, the line of code I used is below.

```
galyleo launch -A cwp157 -p cw3e-shared -t 5-00:00:00 -j lab -B /expanse:/expanse,/scratch:/scratch --sif /expanse/nfs/cw3e/cwp157/cdeciampa/SOFT_ROOT/nwp_env.sif -d /expanse/nfs/cw3e/cwp157/cdeciampa/analysis/notebooks --env-modules singularitypro/4.1.2
```

## Summary 
---
This notebook explained how to build and use a Singularity container containing a micromamba environment.